# 一些探索性的数据分析
## 本文将做
- mobile01 论坛中关于新冠肺炎的讨论热度变化
- 描绘特定用户的观点画像

In [1]:
# !pip install pandas pyecharts hanlp

In [2]:
import re
import sys
import json
import pandas as pd
from pyecharts.charts import Line, WordCloud
from pathlib import Path

In [3]:
# datap = Path('datas')
datap = Path('datas')


In [4]:
# 查看样本
posts = []
for idx, postp in enumerate(datap.iterdir()):
    if postp.suffix == '.json':
        with open(postp) as file:
            posts.append(json.load(file))
        if len(posts[-1]['dialogues'])<2:
            break
posts[-1]

{'topic_id': '780',
 'post_id': '6049068',
 'name': '武漢肺炎》巴西總統之子狂嗆反中言論 惹怒中國大使',
 'create_date': '2020-03-19',
 'resp_cnt': 0,
 'dialogues': {'1': {'author_id': '3263659',
   'author_name': 'maxwu123',
   'content_id': 'article_76889541',
   'content': '2020-03-19 14:30:01\n〔即時新聞／綜合報導〕武漢肺炎疫情全球大流行，繼美國總統川普將病毒稱作「中國病毒」，引發中國不滿後，巴西總統之子、參議員愛德華多（Eduardo Bolsonaro）在社群網站上指中國是「獨裁政權」，並稱病毒爆發是中國的錯。中國駐巴西大使楊萬明不爽回嗆，「你的言論是對中國和中國人民的惡毒攻擊，我們絕不接受。」\n\n愛德華多19日推特引用蘇聯的車諾比核災來比喻，稱中國爆發的武漢肺炎就像是車諾比核災，選擇隱瞞事實，卻不願曝光真相拯救無數性命。愛德華多將中國稱作「獨裁政權」說，「這是中國的錯，自由才是解決之道。」\n\n愛德華多的言論引發中國不滿，中國駐巴西大使楊萬明回應說，「你的言論是對中國和中國人民的惡毒攻擊，我們絕不接受。你作為聯邦眾議員和有特殊身份的公眾人物，發表此番反華言論，非常不妥，將給中巴兩國友好帶來傷害。我對你的推文表示嚴厲譴責，要求你撤銷此條推文，並向中國人民道歉。我將向巴西外交部和眾議院表達中方的嚴重不滿和抗議。」',
   'time': '2020-03-19 16:40'}}}

In [5]:
# 定义与新冠肺炎有关的词语
# 本文只做简单探索，对问题的定义也做了简化
covid_19_related = ['武漢', '病毒', '肺炎', '疫情', '防疫', 'covid', 'cov', 'coronavirus', 'virus', 'sars']

## 以post维度看

In [6]:
# 构建post 维度的数据表
posts = {}
reg = re.compile('\n+')
post_cnt, related_post_cnt = 0, 0
for idx, postp in enumerate(datap.iterdir()):
    if postp.suffix == '.json':
        with open(postp) as file:
            post = json.load(file)
        tmp = {}
        for k in ['topic_id','post_id','name','create_date','resp_cnt']:
            tmp[k] = post[k]
        tmp['main_content'] = reg.sub('\n', post.get('dialogues').get('1', {}).get('content','').lower())
        tmp['if_related'] = any(i in tmp['main_content'] for i in covid_19_related)
        posts[idx] = tmp


In [7]:
post_dataframe = pd.DataFrame.from_dict(posts, orient='index')


In [8]:
post_dataframe.head()

,topic_id,post_id,name,create_date,resp_cnt,main_content,if_related
0,780,6049592,面子重於生命、表演重於防疫！中國式抗疫 國共兩黨皆然,2020-03-20,9,面子重於生命、表演重於防疫！中國式抗疫 國共兩黨皆然\n朱孟庠\n發布 2020.03.19...,True
1,638,5998618,大家覺得我這幾天的表現還可以嗎？？^_^,2020-01-08,6,是否已經足夠讓長輩們體會到我的一片心意了呢？？大選倒數只剩最後3天，我想好好沉澱一下心情，0...,False
2,638,6056133,民眾黨想撤換陳時中，是眼紅還是蹭熱度？,2020-03-28,9,民眾黨想撤換陳時中，是眼紅還是蹭熱度？\n2020-03-27 14:34\n推文到plur...,True
3,780,6049068,武漢肺炎》巴西總統之子狂嗆反中言論 惹怒中國大使,2020-03-19,0,2020-03-19 14:30:01\n〔即時新聞／綜合報導〕武漢肺炎疫情全球大流行，繼美...,True
4,638,6023033,"2009年民進黨大罵馬英九消費卷,結果2020年救經濟的絕招居然是消費卷?",2020-02-13,5,摘錄自工商時報:2009年的台灣經濟表現會發現，當我們上半年出口衰退幅度擴大至三成時，我們的...,False


In [9]:
print(f'总体有关新冠肺炎的帖子占比:{post_dataframe.if_related.sum()/len(post_dataframe):.2%}')

总体有关新冠肺炎的帖子占比:42.56%


### 统计时间与话题数量、占比

In [10]:
def stat(df):
    tmp = {'post_cnt': df.if_related.count(),
        'related_post_cnt': df.if_related.sum(),
     }
    tmp['ratio'] = tmp['related_post_cnt'] / tmp['post_cnt']
    return pd.Series(tmp)
stat_dataframe = post_dataframe.groupby('create_date').apply(stat)


In [11]:
stat_dataframe.head()

,post_cnt,related_post_cnt,ratio
create_date,,,
2020-01-01,186.0,0.0,0.000000
2020-01-02,209.0,1.0,0.004785
2020-01-03,135.0,3.0,0.022222
2020-01-04,146.0,2.0,0.013699
2020-01-05,156.0,4.0,0.025641


In [12]:
# 如果用matplotlib 绘图的话将index 转为datetimeindex 是值得的
# stat_dataframe.index = pd.DatetimeIndex(stat_dataframe.index)
# stat_dataframe.ratio.plot()

### 肺炎相关帖子占比走势
- 01-20前，有零星讨论
- 01-21开始，帖子量、占比暴涨
- 1月底到4月初，肺炎话题仍然保持高热度

In [13]:
# 查看肺炎相关帖子占比走势
line = Line().add_xaxis(
    list(stat_dataframe.index)).add_yaxis(
    'ratio', stat_dataframe.ratio, is_symbol_show=False)
line.render_notebook()

In [14]:
line = Line().add_xaxis(
    list(stat_dataframe.index)).add_yaxis(
    'related_post_cnt', stat_dataframe.related_post_cnt, is_symbol_show=False).add_yaxis(
    'post_cnt', stat_dataframe.post_cnt, is_symbol_show=False)
line.render_notebook()

### 中间看一下词频

In [15]:
import hanlp
from collections import Counter

In [16]:
wc = Counter()

In [17]:
# jieba 对繁体中文的支持不是很好，故采用hanlp
# hanlp 运行时输出比较多，给他重定向到文件
std, sys.stdout = sys.stdout, open('std.txt', 'w')
tokenizer = hanlp.load('PKU_NAME_MERGED_SIX_MONTHS_CONVSEG')
for idx, postp in enumerate(datap.iterdir()):
    if postp.suffix == '.json':
        with open(postp) as f:
            post = json.load(f)
            for k, v in post.get('dialogues').items():
                wc.update(tokenizer(v.get('content','').lower()))
    if idx % 1000==0:
        std.write(f'{idx}\n')
sys.stdout =  std

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [18]:
with open('analysis/word_count.json', 'w') as f:
    json.dump(wc, f)

### 词云图
- 台灣、中國、美國是mobile01 论坛网友讨论最多的国家与地区
- 大陸的词频远少于中國，反映了台湾网友对大陆的称呼多为中国而非大陆
- 党派之争也在论坛词频得以反映
- 武漢“喜提”top 词  


In [49]:
# 待删除的常用词
common = '''現在 會 就是 什麼 多 要 ！ wrote 啊 說 就 只 吧 1 2 3 4 5 6 7 8 9 0 ( ) （ ） 」 為 ？ 自己 大 做 讓 你 好 個 跟 ! 。 這 ) 上 與 在 是 看 那 都 「 年 我們 . 嗎 我 - 不 * 
中 沒有 : 又 來 可以 能 想 對 一 有 真 把 用 的 如果 .. ... 還 了 ： ， 才 但 也 到 被 就 去 沒 , 知道 再 、 ? " 給 他 人 和 一個 ( 很 
而 ： ； : ; \n'''.split()

In [65]:
most_common = {}
for i, j in wc.most_common(1000):
    if i.strip() in common or i==' ':
        continue
    most_common[i] = j

In [66]:
word_cloud = WordCloud().add('词频', most_common.items())
word_cloud.render_notebook()

## 以人为维度看
### 考虑某人的所有发帖、关于新冠肺炎发帖、态度，制作画像

In [52]:
dialogues = {}
n = 0
reg = re.compile('\n+')
for postp in datap.iterdir():
    if postp.suffix == '.json':
        with open(postp) as f:
            post = json.load(f)
            for i, k in post['dialogues'].items():
                tmp = {}
                tmp['post_id'] = post['post_id']
                tmp['order'] = i
                tmp['author_id'] = k['author_id']
                tmp['author_name'] = k['author_name']
                tmp['content'] = reg.sub('\n', k.get('content','').lower())
                tmp['time'] = k['time']
                dialogues[n] = tmp
                n += 1
            

In [53]:
diag = pd.DataFrame.from_dict(dialogues, orient='index')

In [54]:
diag.head()

,post_id,order,author_id,author_name,content,time
0,6049592,1,1704659,ebola01,面子重於生命、表演重於防疫！中國式抗疫 國共兩黨皆然\n朱孟庠\n發布 2020.03.19...,2020-03-20 10:35
1,6049592,2,1752801,robin0222,有一天去百貨用餐\n一個清潔人員拿個抹布握住電扶梯的扶手\n另一個拿相機拍張照 然後兩個人就...,2020-03-20 10:44
2,6049592,3,3204191,weiweichiao,可是主管若加強抽檢，會被笑阿....,2020-03-20 10:46
3,6049592,4,1234395,finepupil,你錯了\n早說綠綠執政無能就是找老共、老k、老馬救援\n本篇老共、老k都有了就差老馬,2020-03-20 10:48
4,6049592,5,120187,mpepsi,你站在那整天看清潔人員嗎？\n如果不是\n你怎麼不想想，說不定是他剛擦完了\n但主管沒有拍照...,2020-03-20 10:52


In [55]:
print('''平均帖子回复数:{:.0f}
人均参与讨论数:{:.0f}'''.format(len(diag)/len(diag.post_id.unique()),
                      len(diag)/len(diag.author_id.unique()))
     )

平均帖子回复数:19
人均参与讨论数:48


In [56]:
diag['if_post_related'] = 0
diag.loc[diag.post_id.isin(post_dataframe.loc[post_dataframe.if_related, 'post_id']), 'if_post_related'] = 1
diag['if_related'] = diag.content.apply(lambda x:any(i in str(x) for i in covid_19_related)) | diag.if_post_related

In [57]:
print('''与新冠肺炎有关的回复:{}
占比{:.2%}'''.format(diag.if_related.sum(), diag.if_related.sum()/len(diag)))

与新冠肺炎有关的回复:109331
占比41.82%


### Top Users

In [58]:
authors = diag.groupby('author_id').first()[['author_name']]


In [59]:
top = pd.DataFrame(diag.author_id.value_counts()).join(authors).rename(columns={'author_id': 'diag_cnt'})

In [60]:
top.iloc[:3]

,diag_cnt,author_name
2200484,3637,Enter Passcode
2834082,3114,chiashin
3150424,2754,freedomhome


- 恭喜 Enter Passcode，mobile01 时事区顶级讨论者，进一步看了老哥的主页，时事区还不是他的主场，活跃度max
![](https://wuting-own.oss-cn-beijing.aliyuncs.com/share/d2a943727d4c44dea87b18e661cfb124.png)

In [61]:
top.loc[sorted(top.index, key=lambda x:int(x))[:3],]

,diag_cnt,author_name
1437,6,Mobile01系統服務員
1677,5,AYIYA
2111,1,Rosen


In [62]:
# 看看系统管理员没事在时事区说些什么，然而全都是管理事项，真是个好管理员robot。。。
diag.loc[diag.author_id=='1437', 'content']


15071     各位會員大家好\n確認到此討論串中已產生會員間回文對立及不理性的言論發表，站上已備有黑名單的...
17497     發文者您好, 您的文章因以下事由已設定為唯讀狀態:\n6. 發言涉及攻擊、侮辱、影射或其他有...
34813     發文者您好, 您的文章因以下事由已設定為唯讀狀態:\n6. 發言涉及攻擊、侮辱、影射或其他有...
91696     各位會員大家好：\n很謝謝一直以來的支持，由於程式bug 造成使用上的困擾，深感抱歉。it工...
167840    發文者您好, 您的文章因以下事由已設定為唯讀狀態:\n6. 發言涉及攻擊、侮辱、影射或其他有...
260741    發文者您好, 您的文章因以下事由已設定為唯讀狀態:\n6. 發言涉及攻擊、侮辱、影射或其他有...
Name: content, dtype: object

- 最元老的用户 id 1437, Mobile01系統服務員,兢兢业业
- 最元老的非系统管理员用户 id 1677,AYIYA。看来这位老哥曾经有过黑历史啊
![](https://wuting-own.oss-cn-beijing.aliyuncs.com/share/ea21d24115d04f2687f95b7b1b3df8de.png)

### 接下来挑选一名对新冠肺炎非常上心的朋友

In [63]:
carecovid = pd.DataFrame(diag.groupby('author_id').sum().sort_values('if_related', ascending=False)).join(authors)

In [64]:
carecovid.iloc[:3]

,if_post_related,if_related,author_name
author_id,,,
2834082,1552,1685.0,chiashin
2257617,1308,1404.0,economic
1704659,1194,1196.0,ebola01


- id 2834082,chiashin 这位朋友确实对人类命运有高尚的担当，发布或回复了1300条+关于新冠肺炎的帖子或对话。向您致敬
![](https://wuting-own.oss-cn-beijing.aliyuncs.com/share/31561e8971d94ab7bb23bae62d4499ad.png)

## 数据探索就先到这结束啦
接来下还要探索一个问题：大陆对疫情的对策从掩盖到慌乱应对到有序对抗到压制，这一系列动作在台湾同胞们看来是怎么样的评价呢？还请关注